In [ ]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt

# Read the Excel file into a DataFrame
df = pd.read_excel('Input_Unbanx.xlsx')

# Drop rows where all column values are empty
df.dropna(how='all', inplace=True)

# Function to extract numeric parts with more than 6 digits
def extract_numeric(description):
    numeric_parts = re.findall(r'\b\d+\b', str(description))
    large_numeric_parts = [part for part in numeric_parts if len(part) > 6]
    return large_numeric_parts[0] if large_numeric_parts else None

# Apply function to description column
df['numeric_part'] = df['description'].apply(extract_numeric)

# Filter rows with non-null numeric_part
filtered_df = df[df['numeric_part'].notna()]

# Filter rows with non-null enrichments_merchant_display_name
filtered_df = filtered_df[filtered_df['enrichments_merchant_display_name'].notna()]

# Get unique values of enrichments_merchant_display_name
unique_values = filtered_df['enrichments_merchant_display_name'].unique()

# Save subsets to CSV files
for value in unique_values:
    filename = re.sub(r'[\\/*?:"<>|]', '_', str(value)) + '.csv'
    subset_df = filtered_df[filtered_df['enrichments_merchant_display_name'] == value]
    subset_df.to_csv(filename, index=False)

# Add a new column 'merchant_check'
filtered_df['merchant_check'] = filtered_df['enrichments_merchant_display_name'].apply(lambda x: x if x in unique_values else None)

# Create a pivot table
pivot_table = filtered_df.pivot_table(index='provider', columns='merchant_check', aggfunc='size', fill_value=0)

# Plot heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(pivot_table, annot=True, fmt='d', cmap='YlGnBu')
plt.title('Count of Merchant Checks by Provider')
plt.xlabel('Merchant Check')
plt.ylabel('Provider')
plt.show()

# Plot clustered bar chart
plt.figure(figsize=(12, 6))
sns.countplot(data=filtered_df, x='provider', hue='merchant_check')
plt.title('Count of Merchant Checks by Provider')
plt.xlabel('Provider')
plt.ylabel('Count')
plt.legend(title='Merchant Check', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Plot heatmap with adjusted color scale
plt.figure(figsize=(12, 8))
sns.heatmap(pivot_table, cmap='YlGnBu', linewidths=0.5, linecolor='grey', annot=True, fmt='d', cbar_kws={'label': 'Count'})
plt.title('Count of Merchant Checks by Provider')
plt.xlabel('Merchant Check')
plt.ylabel('Provider')
plt.show()
